In [1]:
%load_ext autoreload
%autoreload 2
import time

In [2]:
import design as d
import design_variables as dv
import src.utils.utils_design_variables as u
from src.utils.utils_design import create_chip_base

# Design assembly

In [3]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
CHIP_TYPE = {"size_x": "10mm",
             "size_y": "10mm",
             "size_z": "-300um"}
design, gui = create_chip_base(chip_name=CHIP_NAME, chip_type=CHIP_TYPE, open_gui=OPEN_GUI)

u.add_design_variables_to_design(design, dv.DESIGN_VARS)

In [4]:
def render_qiskit_metal_design(design, gui):
    d.add_branch(design, 0, gui)
    d.add_branch(design, 1, gui)
    d.add_branch(design, 2, gui)
    d.add_branch(design, 3, gui)
    d.add_branch(design, 4, gui)

    d.add_route_interconnects(design, 0, gui)
    d.add_route_interconnects(design, 1, gui)
    d.add_route_interconnects(design, 2, gui)
    d.add_route_interconnects(design, 3, gui)
    
    d.add_launch_pads(design, gui)
render_qiskit_metal_design(design, gui)

# MiniStudies

In [ ]:
import mini_studies as ms
import optimization_targets as ot
import target_parameters as tp
import plot_settings as ps

from src.qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState

In [ ]:
MINI_STUDY_BRANCH = 0
MINI_STUDY = ms.get_mini_study_qb_res(branch=MINI_STUDY_BRANCH)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

################# optimization targets ##############
opt_targets = []
for target in ot.get_opt_targets_qb_res(
        branch=MINI_STUDY_BRANCH,
    ):
    opt_targets.append(target)

In [ ]:
design_analysis_state = DesignAnalysisState(design, RENDER_QISKIT_METAL, tp.TARGET_PARAMS)
design_analysis = DesignAnalysis(
    design_analysis_state, 
    mini_study=MINI_STUDY, 
    opt_targets=opt_targets,
    print_progress=True,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    plot_settings=ps.PLOT_SETTINGS,
    plot_branches_separately=False
    )

nbr_runs = 10
nbr_passes = 4
delta_f = 0.001
for i in range(nbr_runs):   
    design_analysis.update_nbr_passes(nbr_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})

In [ ]:
from src.utils.utils import close_ansys
close_ansys()